## 16. 코루틴
### 16.2 코루틴으로 사용되는 제너레이터의 기본 동작

예제 16-1은 코루틴의 동작을 보여준다.

In [1]:
""" [예제 16-1] 가장 간단한 코루틴 사용 """
from inspect import getgeneratorstate

def simple_coroutine(): # 코루틴은 자신의 본체 안에 yield문을 가진 일종의 제너레이터 함수로 정의된다.
    print('-> coroutine started')
    x = yield # yield를 표현식에 사용한다. 단지 호출자에 데이터를 받도록 설계하면 yield는 값을 생성하지 않는다.
              # yield 키워드 뒤에 아무런 표현식이 없을 때 값을 생성하지 않으려는 의도를 암묵적으로 표현한다.
    print('-> coroutine received:', x)
    
my_coro = simple_coroutine() # 일반적인 제너레이터와 마찬가지로 함수를 호출해서 제너레이터 객체를 가져온다.
print(repr(my_coro))

<generator object simple_coroutine at 0x7f044049ac50>


In [2]:
print(getgeneratorstate(my_coro))
next(my_coro) # next()를 호출해서 제너레이터를 yield문 까지 실행함으로써 데이터를 전송할 수 있는 상태를 만든다.
              # my_coro.send(None)을 호출해도 효과가 동일하다.
print(getgeneratorstate(my_coro))

GEN_CREATED
-> coroutine started
GEN_SUSPENDED


In [3]:
print(getgeneratorstate(my_coro))
my_coro.send(42) # 제너레이터의 send() 메서드를 호출해서 코루틴 본체 안의 yield 문의 값을 42로 만든다.
                 # 이제 코루틴이 실행을 재개해서 다음 yield문이 나오거나 종료될 때까지 실행한다.
                 # 제어흐름이 코루틴 본체의 끝에 도달하므로, 일반적인 제너레이터와 마찬가지로 StopIteration 예외를 발생시킨다.

GEN_SUSPENDED
-> coroutine received: 42


StopIteration: 

In [4]:
print(getgeneratorstate(my_coro))

GEN_CLOSED


코루틴은 네 가지 상태를 가진다. inspect.getgeneratorstate( ) 함수를 이용해서 현재 상태를 알 수 있다.
+ GEN_CREATE    : 실행을 시작하기 위해 대기하고 있는 상태
+ GEN_RUNNING   : 현재 인터프리터가 실행하고 있는 상태
+ GEN_SUSPERNED : yield문에서 대기하고 있는 상태
+ GEN_CLOSED    : 실행이 완료된 상태

코루틴 객체를 생성하고 난 직후(GEN_CREATE 상태)에 바로 None이 아닌 값을 전달하려고 하면 다음과 같은 오류가 발생한다. 처음 next(my_coro)를 호출할 때, 코루틴을 기동<sub>priming</sub> 한다고 표현한다. 즉, 코루틴이 호출자로부터 값을 받을 수 있도록 처음 yield문까지 실행을 진행하는 것이다.

In [5]:
my_coro = simple_coroutine()
my_coro.send(1729)

TypeError: can't send non-None value to a just-started generator

In [6]:
""" [예제 16-2] 두 번 생성하는 코루틴 """

def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

my_coro2 = simple_coro2(14)
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [7]:
next(my_coro2)

-> Started: a = 14


14

In [8]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [9]:
my_coro2.send(42)

-> Received: b = 42


56

In [10]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [11]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

[그림 16-1] simple_coro2 코루틴을 실행하는 3 단계. 각 단계는 yield 표현식에서 끝나며, 다음 단계는 yield 표현식의 값을 변수에 할당하는 동일 행에서 시작한다. 
<img src="Figure16-1.png">

### 16.3 이동 평균을 계산하는 코루틴
예제 7-14에서 클로저를 생성해서 total과 count 변수를 보전하는 고급함수와 비교해보자.

In [12]:
""" [예제 7-14] nonlocal 키워드로 오류 해결 """
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    
    return averager

avg = make_averager() 
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

In [13]:
""" [예제 16-3, 4] 이동 평균 코루틴과 doctest """
def averager():
    count = 0
    total = 0
    average = None
    
    while True: # 무한 루프이므로 이 코루틴은 호출자가 값을 보내주는 한 계속해서 값을 받고 결과를 생성한다.
                # 이 코루틴은 호출자가 close() 메서드를 호출하거나,
                # 이 객체에 대한 참조가 모두 사라져서 가비지 컬렉트되어야 종료된다. 
        term = yield average # 이 yield 문은 코루틴을 중단하고 지금까지의 평균을 생성하기 위해 사용된다. 
        total += term
        count += 1
        average = total/count

coro_avg = averager()
next(coro_avg)

In [14]:
coro_avg.send(18)

18.0

In [15]:
coro_avg.send(30)

24.0

In [16]:
coro_avg.send(5)

17.666666666666668

### 16.4 코루틴을 기동하기 위한 데커레이터
코루틴을 편리하게 사용할 수 있도록 기동하는 데커레이터가 종종 사용된다. @coroutine가 대표적이다.

In [17]:
""" [예제 16-5] 코루틴을 기동하기 위한 데커레이터. (yield from과 함께 사용 불가능) """
from functools import wraps

def coroutine(func):
    """ 데커레이터: 'func'를 기동해서 첫 번째 'yield'까지 진행한다. """
    @wraps(func)                    # 데커레이트된 제너레이터 함수는 primer() 함수로 치환되며, 실행하면 기동된 제너레이터를 반환한다.
    def primer(*args, **kwargs):    # 데커레이트된 함수를 호출해서 제너레이터 객체를 가져온다.
        gen = func(*args, **kwargs) 
        next(gen)                   # 제너레이터를 기동한다.
        return gen                  # 제너레이터를 반환한다.
    return primer

In [18]:
@coroutine
def averager():
    count = 0
    total = 0
    average = None
    
    while True:
        term = yield average 
        total += term
        count += 1
        average = total/count

In [19]:
coro_avg = averager() # 생성만으로 코루틴의 상태가 대기(suspended) 상태이다.

from inspect import getgeneratorstate
getgeneratorstate(coro_avg) 

'GEN_SUSPENDED'

In [20]:
coro_avg.send(18)

18.0

In [21]:
coro_avg.send(30)

24.0

In [22]:
coro_avg.send(5)

17.666666666666668

### 16.5 코루틴 종료와 예외 처리
코루틴 안에서 발생한 예외를 처리하지 않으면, next( )나 send( )로 코루틴을 호출한 호출자에 예외가 전파된다.

In [23]:
""" [예제 16-7] 처리하지 않은 예외에 의한 코루틴 종료 """
coro_avg = averager()
coro_avg.send(40)
coro_avg.send(50)

45.0

In [24]:
coro_avg.send('spam') # 코루틴 내에서 예외 발생

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

In [25]:
coro_avg.send(60) # 코루틴 안에서 예외를 처리하지 않으면 루프가 종료되고 다시 활성화되지 않는다.

StopIteration: 

[예제 16-7]을 보면 종료하라고 코루틴에 알려주는 구분 표시를 전송해서 코루틴을 종료할 수 있음을 알 수 있다. None이나 Eliipsis와 같은 내장된 싱글턴 상수는 구분표시로 사용하기 좋다. (필자는 my_coro.send(StopIteration) 형태로 사용하기도 한다.)

파이선 2.5 이후 제너레이터 객체는 호출자가 코루틴에 명시적으로 예외를 전달할 수 있도록 throw( )와 close( ) 메서드를 제공한다.

```
generator.throw(exc_type[, exc_value[, traceback]])
제너레이터가 중단한 곳의 yield 표현식에 예외를 전달한다. 제너레이터가 예외를 처리하면 제어흐름이 다음 yield 문까지 진행하고 생성된 값은 generator.throw() 호출 값이 된다. 제너레이터가 예외를 처리하지 않으면 호출자까지 예외가 전파된다.
```
```
generator.close( )
제너레이터가 실행을 중단한 yield 표현식이 GeneratorExit 예외를 발생시키게 만든다. 제너레이터가 예외를 처리하지 않거나 StopIteration 예외(일반적으로 제너레이터가 실행을 완료할 때 발생한다.)를 발생시키면 아무런 에러도 호출자에 전달되지 않는다. GeneratorExit 예외를 받으면 제너레이터는 아무런 값도 생성하지 않아야 한다. 아니면 RuntimeError 예외가 발생한다. 제너레이터에서 발생하는 다른 예외는 모두 호출자에 전달된다.
```

In [26]:
""" [예제 16-8] 코루틴의 예외 처리방법을 설명하기 위한 제너레이터 """

class DemoException(Exception):
    """ 설명에 사용할 예외 유형 """
    
def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException: # 예외를 따로 처리
            print('*** DemoException handled. Continuing... ')
        else: # 예외가 발생되지 않으면 받은 값을 출력
            print('-> coroutine received: {!r}'.format(x))
#     raise RuntimeError('This line should never run.') # 무한 루프는 처리되지 않은 예외에 의해서만 중단될 수 있으며,
#                                                       # 예외를 처리하지 않으면 코루틴의 실행이 바로 중단되므로 마지막 줄은 실행될 수 없다.
#                                                       # 책과는 다르게 인터프리터가 마지막 줄은 실행될 수 없음을 알림(Upgrade 된듯)

In [27]:
""" [예제 16-9] 예외를 발생시키지 않은 demo_exc_handling의 활성화 및 종료 """

exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [28]:
exc_coro.send(11)

-> coroutine received: 11


In [29]:
exc_coro.send(22)

-> coroutine received: 22


In [30]:
exc_coro.close() # GeneratorExit 예외를 발생시키며 제너레이터는 아무런 값을 생성하지 않는다. 

In [31]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [32]:
""" [예제 16-10] DemoException을 demo_exc_handling 안에 던져도 종료되지 않음 """

exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [33]:
exc_coro.send(11)

-> coroutine received: 11


In [34]:
exc_coro.throw(DemoException) # 예외를 던졌으나

*** DemoException handled. Continuing... 


In [35]:
getgeneratorstate(exc_coro)  # 종료되지 않았음

'GEN_SUSPENDED'

In [36]:
""" [예제 16-11] 자신에게 던져진 예외를 처리할 수 없으면 코루틴이 종료됨 """
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.send(11)
exc_coro.throw(ZeroDivisionError)

-> coroutine started
-> coroutine received: 11


ZeroDivisionError: 

In [37]:
getgeneratorstate(exc_coro) # ZeroDivisionError는 처리할 수 없으므로 종료됨

'GEN_CLOSED'

코루틴이 어떻게 종료되든 정리 코드를 실행해야 하는 경우에는 try/finally 블록 안에 코루틴의 해당 코드를 넣어야 한다.

In [38]:
""" [예제 16-12] 코루틴 종료 시 정리작업을 실행하기 위핸 try/finally문 추가 """

class DemoException(Exception):
    """ 설명에 사용할 예외 유형 """
    
def demo_finally():
    print('-> coroutine started')
    
    try:
        while True:
            try:
                x = yield
            except DemoException: # 예외를 따로 처리
                print('*** DemoException handled. Continuing... ')
            else: # 예외가 발생되지 않으면 받은 값을 출력
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

In [39]:
exc_coro = demo_finally()
next(exc_coro)
exc_coro.throw(ZeroDivisionError) # 처리할 수 없는 에러를 받는 경우 finally 문이 실행된다.

-> coroutine started
-> coroutine ending


ZeroDivisionError: 

### 16.6 코루틴에서 값 반환하기
예제 16-13은 항목 수(count)와 평균(average)을 담은 namedtuple을 반환한다.

In [40]:
""" [예제 16-13] averager( ) 코루틴 코드 """

from collections import namedtuple
Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break # 값을 반환하려면 코루틴이 정상적으로 종료되어야 한다. 그렇기 때문에 averager 버전에서는 루프를 빠져나오는 조건을 검사한다.
        total += term
        count += 1
        average = total/count
    return Result(count, average) # 파이선 3.3 이전 버전에서는 제너레이터가 값을 반환하므로 에러가 발생한다.  

In [41]:
""" [예제 16-14] averager( )의 동작을 보여주는 doctest """

coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg.send(None) # 일반적인 제너레이터 객체와 같이 StopIteration 예외가 발생하며 value 속성에 반환값이 들어 있다.

StopIteration: Result(count=3, average=15.5)

In [42]:
""" [예제 16-15] StopIteration 예외 처리를 통한 반환값 가져오기 """

coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)

try: # try문을 통한 우회적인 처리 방법 (StopIteration이 자연스럽게 처리되었다.)
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
    
print(result)

Result(count=3, average=15.5)


### 16.7 yield from 사용하기
yield from은 완전히 새로운 언어 구성체라는 점을 잊지말자. (다른 언어에서는 이와 비슷한 구성체를 await라고 표현한다.) 

yield from x 표현식이 x 객체에 대해 첫 번째로 하는 일은 iter(x)를 호출해서 x의 반복자를 가져오는 것이다. 이는 모든 반복형이 x에 사용될 수 있다는 의미이다.

In [43]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i
        
list(gen())

['A', 'B', 1, 2]

In [44]:
def gen():
    yield from 'AB'
    yield from range(1, 3)
        
list(gen())

['A', 'B', 1, 2]

그러나 내포된 for 루프를 대체하는 것보다 더 주요한 특징은 가장 바깥쪽 호출자와 가장 안쪽 하위 제너레이터 사이에 양방향 채널을 열어준다는 것이다. 따라서 이 둘이 값을 직접 주고 받으며 중간에 있는 코루틴이 판에 박힌 듯한 예외처리 코드를 구현할 필요 없이 예외를 직접 던질 수 있다.이를 코루틴 위임<sup>coroutine delegation</sup>이라 한다.

아래는 중요 용어에 대한 설명이다.
+ 대표 제너레이터(delegating generator)
    + yield from <반복형> 표현식을 담고 있는 제너레이터 함수
+ 하위 제너레이터(subgenerator)
    + yield from 표현식 중 <반복형>에서 가져오는 제너레이터
+ 호출자(caller)
    + 대표 제너레이터를 호출하는 코드. 문맥에 따라서 필자는 대표 제너레이터와 구분하기 위해 '호출자' 대신 '클라이언트'라는 용어를 사용하기도 한다. 하위 제너레이터 입장에서 보면 대표 제너레이터도 호출자이기 때문이다.
    
[그림 16-2] 대표 제너레이터가 yield from에서 중단하고 있는 동안, 호출자는 하위 제너레이터에 데이터를 직접 전송하고 하위 제너레이터는 다시 데이터를 생성해서 호출자에게 전달한다. 하위 제너레이터가 실행을 완료하고 인터프리터가 반환된 값을 첨부한 StopIteration을 발생시키면 대표 제너레이터가 실행을 재개한다.

<img src="Figure16-2.png">

In [45]:
""" [예제 16-17] yield from을 이용해서 averager()를 구동하고 보고서 생성하기 """

from collections import namedtuple

Result = namedtuple('Result', 'count average')

# 하위 제너레이터
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield     # term에 데이터를 받을 수 있음
        if term is None: # StopIteration가 발생되면서 대표 제너레이터의 동작이 재개되는 지점
            break        # 이 부분이 없으면 영원히 실행된다.
        total += term
        count += 1
        average = total/count
    return Result(count, average)

# 대표 제너레이터
def grouper(results, key):
    while True: # 반복할 때 마다 averager 객체를 만든다
        results[key] = yield from averager()

# 호출자(클라이언트)
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key) # 대표제너레이터에 계산한 값을 받을 result와, key 정보를 전달한다.
        next(group) # 코루틴을 기동한다.
        for value in values:  
            group.send(value) # 하위 제너레이터와 통로가 생겼으니 값을 하나씩 하위 제너레이터인 grouper()에 전달한다. 이 값은 averager() term = yield 문장의 yield 값이 된다.
                              # grouper는 이 값을 볼 수 없다.
        group.send(None) # (이 부분이 중요하다!) None을 grouper()에 전달하면 현재 averager() 객체가 종료된다.
                         # 이후 다음 루트에서 그러면 grouper()는 또 다른 averager() 객체를 생성해서 다음 값들을 연산한다.
                         # 즉, 한 아이템당 한 루프가 돌면서 제너레이터가 생성되고 연산 후 사라진다. 
    # print(results) # for debug
    report(results)
    
# 실행결과 보고서
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'
              .format(result.count, group, result.average, unit))

data = {
    'girls;kg': [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m' : [ 1.6, 1.51,  1.4,  1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg' : [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m'  : [1.38,  1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],    
} # group과 unit을 ; 로 구분하고 있음

if __name__ == '__main__':
    main(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


예제 16-17 에서 group.send(None)을 제거하면 아래와 같은 현상이 생긴다.
+ next(group)을 호출해서 grouper() 대표 제너레이터를 기동시키면 while True 루프로 들어가서 하위 제너레이터 averager()를 호출한 후 yield from에서 대기한다.
+ 내부 for 루프에서 group.send(value)를 호출해서 하위 제너레이터 averager()에 직접 데이터를 전달한다. 이때 grouper()의 현재 group 객체는 여전치 yield from에서 멈추게 된다.
+ 내부 for 루프가 끝났을 때 grouper() 객체는 여전히 yield from에 멈워있으므로, grouper() 본체 안의 results[key]에 대한 할당은 아직 실행되지 않는다.
+ 바깥쪽 for 루프에서 group.send(None)을 호출하지 않으면 하위 제너레이터인 averager()이 계속 루프를 돌고 있는 상황(yield 멈춰있긴 함)이므로 results[key]에 아무런 값도 할당되지 않는다. 
+ 바깥쪽 for 루프의 꼭대기로 올라가서 다시 반복하면 새로운 grouper() 객체가 생성되어 group 변수에 바인딩된다. 기존 grouper() 객체는 더 이상 참조되지 않으므로 가비지 컬렉드된다. 이때 아직 실행이 종료되지 않은 averager() 하위 제너레이터 객체도 지워진다. 

### 16.8 yield from 의 의미

[예제 16-7]은 다음 네 가지 특징을 보여준다.
+ 하위 제너레이터가 생성하는 값은 모두 대표 제너레이터의 호출자(클라이언트)에 바로 전달된다. 
+ send( )를 통해 대표 제너레이터에 전달한 값은 모두 하위 제너레이터에 직접 전달된다. 값이 None이면 하위 제너레이터의 \_\_next\_\_( ) 메서드가 호출된다. 전달된 값이 None이 아니면 하위 제너레이터의 send( ) 메서드가 호출된다. 호출된 메서드에서 StopIteration 예외가 발생하면 대표 제너레이터의 실행이 재개된다. 그 외의 예외는 대표 제너레이터에 전달된다. 
+ 하위 제너레이터에서 return expr 문을 실행하면, 제너레이터를 빠져나온 후 StopIteration(expr) 예외가 발생한다. 
+ 하위 제너레이터가 실행을 마친 후 발생한 StopIteration 예외의 첫 번째 인수가 yield from 표현식의 값이 된다. 

yeild from의 나머지 특징 두 가지는 예외와 종료에 관련되어 있다.
+ 대표 제너레이터에 던져진 GeneratorExit 이외의 예외는 하위 제너레이터의 throw() 메서드에 전달된다. 이 때 StopIteration 예외가 발생되면 대표 제너레이터의 실행이 제개되지만 그 외의 예외는 대표 제너레이터에 전달된다. 
+ GeneratorExit 예외가 대표 제너레이터에 던져지거나 대표 제너레이터의 close() 메서드가 호출되면 하위 제너레이터의 close() 메서드가 호출된다. 그 결과 예외가 발생하면 발생한 예외가 대표 제너레이터에 전파된다. 그렇지 않으면 GeneratorExti 예외가 발생한다.

아래 의사코드는 대표 제너레이터에서 "RESULT = yield from EXPR" 문장 하나를 확장한 것이다.
```
_i(iterator) : 하위 제너레이터
_y(yielded)  : 하위 제너레이터가 생성한 값
_r(result)   : 최종 결과값(즉, 하위 제너레이터가 종료된 후 yield from 표현식의 값)
_s(sent)     : 호출자가 대표 제너레이터에 보낸 값
_e(exception): 예외
```

```
# [예제 16-8] throw()와 close() 메서드를 지운 간단 버전
_i = iter(EXPR)   # 특정 하위 제너레이터가 _i에 매핑된다.
try:
    _y = next(_i) # 하위 제너 레이터를 기동한다.
except StopIteration as _e(예외) :
    _r = _e.value
else:
    while 1: # 이 루프를 실행하는 동안 대표 제너레이터는 실행이 중단되고 단지 호출자와 하위 제너레이터 간의 통로 역할만 수행한다.
        _s = yield _y # 
        try:
            _y = _i.send(_s)
        except StopIteration as _e:
            _r = _e.value
            break
RESULT = _r
```


### 16.9  사용 사례: 이산 이벤트 시뮬레이션을 위한 코루틴
#### 16.9.1 이산 이벤트 시뮬레이션에 대해
이산 이벤트 시뮬레이션(DES<sup>discrete event simulation</sup>)은 시스템을 일련의 이벤트로 모델링한다. DES에서 시뮬레이션 '시계'는 고정된 값만큼 진행하는 것이 아니라 모델링된 다름 이벤트의 시뮬레이션된 시각으로 바로 진행한다. 턴제 게임을 떠올리면 쉽다. 게임 상태는 플레이어가 말을 움직일 때만 변경되고, 어디로 움직일지 고민하는 동안 시뮬레이션 시계는 멈춘다. 코루틴은 DES를 작성하기 위한 추상화에 적절하다. Simpy는 시뮬레이션의 각 프로세스를 표현하기 위해 하나의 코루틴을 사용하는 파이선용 DES 패키지이다. 

#### 16.9.2 택시 집단 시뮬레이션
각 택시는 일정 횟수의 운행을 마친 후 집을 돌아간다. 택시는 차고를 나와 승객을 찾으면서 배회한다. 이 상태는 승객을 태울 때까지 계속되며, 승객을 태우면 운행이 시작된다. 승객이 내리고 나면, 택시는 다시 배회 상태로 들어간다. 택시가 배회하고 운행하는 시간은 지수 분포를 이용한다.

※ 사건이 서로 독립적일 때, 일정 시간동안 발생하는 사건의 횟수가 푸아송 분포를 따른다면, 다음 사건이 일어날 때까지 대기 시간은 지수분포를 따른다.
<img src = https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Exponential_pdf.svg/270px-Exponential_pdf.svg.png> <img src = https://upload.wikimedia.org/wikipedia/commons/thumb/b/ba/Exponential_cdf.svg/270px-Exponential_cdf.svg.png>

In [46]:
""" [예제 A-6] 택시 집단 시뮬레이터 """

import random
import collections
import queue
import argparse
import time

DEFAULT_NUMBER_OF_TAXIS = 3
DEFAULT_END_TIME = 180
SEARCH_DURATION = 5
TRIP_DURATION = 20
DEPARTURE_INTERVAL = 5

""" 
time : 이벤트가 발생한 시각
proc : 택시 프로세스 객체의 식별자
action : 행동을 설명하는 문자 
"""
Event = collections.namedtuple('Event', 'time proc action')

In [47]:
# TAXI_PROCESS
def taxi_process(ident, trips, start_time=0):
    """
    각 상태 변화마다 이벤트를 발생시키는 시뮬레이터에 양보한다
        - ident : 택시 번호
        - trip  : 집에 돌아가기 전에 이 택시가 수행할 운행 횟수
        - start_time : 차고를 나오는 시간
    """
    time = yield Event(start_time, ident, 'leave garage') # yield에서 중단되고 클라이언트가 갱신된 시간을 보내주면 택시의 다음 이벤트를 보내준다.
    for i in range(trips): # 택시의 운행 횟수만큼 반복한다. 
        time = yield Event(time, ident, 'pick up passenger') 
        time = yield Event(time, ident, 'drop off passenger')
    yield Event(time, ident, 'going home') # 재활성화될 때 클라이언트가 시간을 보내지만, 사용하지 않으므로 저장하지는 않는다. 
    
# TAXI_SIMULATOR
class Simulator:
    def __init__(self, procs_map):
        """
        이벤트 큐와 프로세스 맵을 초기화
            - event : PriorityQueue() 객체, 여기서는 time 속성으로 정렬된 순서로 꺼내온다. 
            - procs : 택시 하나를 나타내는 제너레이터 객체를 하나의 프로세스로 표현
        """
        self.events = queue.PriorityQueue() # 예정된 이벤트를 시간 순으로 정렬해서 보관
        self.procs = dict(procs_map)        # 사본을 만든다. 집으로 돌아가는 택시들이 run() 마지막에 제거되지만 클라이언트의 객체도 지워지는 것을 방지한다.
        
    def run(self, end_time):
        """ 시간이 만료될 때까지 이벤트를 스케줄링하고 출력한다 """
        
        # 1. 택시를 나타내는 프로세스를 가동하고, 처음 이벤트를 큐에 입력함
        for _, proc in sorted(self.procs.items()): # 3개의 taxi_process가 시작됨. 키로 정렬된 순서로 가져온다.
            first_event = next(proc) # procs는 taxi_processes를 의미하며 모든 제너레이터를 가동시킴
            self.events.put(first_event)
            
        # 2. sim_time < end_time인 동안 시뮬레이션 핵심 루프를 실행 
        sim_time = 0
        while sim_time < end_time: 
            
            # a. self.event가 비어 있으면 종료
            if self.events.empty():
                print('*** end of events ***')
                break
            
            # b. PriorityQueue에서 가장 작은 time 값을 가진 event를 가져와서 출력한다.
            current_event = self.events.get()
            sim_time, proc_id, previous_action = current_event # 현재 이벤트의 시간과, 택시 번호, 이전 이벤트의 액션을 대입
            print('taxi:', proc_id, proc_id * '   ', current_event)
            
            # c. 해당 이벤트의 택시 프로세스를 가져와서 다음 시뮬레이션 시간을 설정한다.
            active_proc = self.procs[proc_id] 
            next_time = sim_time + compute_duration(previous_action) # 현재 시간에서 이벤트에 소요되는 시간을 더해서 다음 이벤트의 시간을 구함
            
            # d. current_event의 proc 속성으로 알아낸 코루틴에 시각을 보낸다. 그러면 코루틴이 next_event를 생성한다.
            try:
                next_event = active_proc.send(next_time) 
            except StopIteration: # 프로세스가 종료되어 다음 이벤트가 없는 경우는 해당 프로세스를 삭제한다.
                del self.procs[proc_id]
            # e. next_event를 큐에 추가해서 스케줄링한다.
            else: # next_event를 성공적으로 받아오는 경우 큐에 보관함.
                self.events.put(next_event)
        else: # 이벤트가 소진된 경우가 아니라 end_time에 도달한 경우이므로 별도의 메시지를 출력한다.
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))

In [48]:
def compute_duration(previous_action):
    """ 지수 분포를 이용해서 행동 기간을 계산한다 """
    print(previous_action)
    if previous_action in ['leave garage', 'drop off passenger']:
        # 손님없이 배회하는 상태
        interval = SEARCH_DURATION
    elif previous_action == 'pick up passenger':
        # 손님을 태우고 운행하는 상태
        interval = TRIP_DURATION
    elif previous_action == 'going home':
        interval = 1
    else:
        raise ValueError('Unknow previous_action: {}'.format(previous_action))
    return int(random.expovariate(1/interval)) + 1

def main(end_time=DEFAULT_END_TIME, num_taxis=DEFAULT_NUMBER_OF_TAXIS, seed=None):
    if seed is not None:
        random.seed(seed) # 동일한 시드는 동일한 시뮬레이션 결과를 가져온다
    taxis = { i:taxi_process(i, (i+1)*2, i*DEPARTURE_INTERVAL) # 0, 2, 0
              for i in range(num_taxis) }                      # 1, 4, 5
                                                               # 2, 6, 10 으로 택시가 생성됨
    sim = Simulator(taxis)
    sim.run(end_time)

if __name__ == '__main__':
    main(num_taxis=2, seed=10)

taxi: 0  Event(time=0, proc=0, action='leave garage')
leave garage
taxi: 0  Event(time=5, proc=0, action='pick up passenger')
pick up passenger
taxi: 1     Event(time=5, proc=1, action='leave garage')
leave garage
taxi: 1     Event(time=10, proc=1, action='pick up passenger')
pick up passenger
taxi: 1     Event(time=15, proc=1, action='drop off passenger')
drop off passenger
taxi: 0  Event(time=17, proc=0, action='drop off passenger')
drop off passenger
taxi: 1     Event(time=24, proc=1, action='pick up passenger')
pick up passenger
taxi: 0  Event(time=26, proc=0, action='pick up passenger')
pick up passenger
taxi: 0  Event(time=30, proc=0, action='drop off passenger')
drop off passenger
taxi: 0  Event(time=34, proc=0, action='going home')
going home
taxi: 1     Event(time=46, proc=1, action='drop off passenger')
drop off passenger
taxi: 1     Event(time=48, proc=1, action='pick up passenger')
pick up passenger
taxi: 1     Event(time=110, proc=1, action='drop off passenger')
drop off p



+ 각 택시는 앞 차가 출발한지 5분 후에 차고를 출발한다. 
+ 첫 승객을 타우기까지 택시 0은 2분(time=2), 택시 1은 3분(time=8), 택시 2는 5분(time=15) 걸렸다.
+ 택시 0은 두 번의 운행을 했다. 첫 번째 운행은 time=2에 시작해서 time=18에 끝났고, 두 번째는 time=28에 시작해서 time=65에 끝났다(이 시뮬레이셔네서 가장 긴 운행)
+ 택시 1은 네 번의 운행을 하고 time=110에 집으로 돌아갔다.
+ 택시 2는 여섯 번의 운행을 하고 time=109에 집으로 돌아갔다. 마지막 운행은 time=97에서 시작해서 1분 만에 끝났다.
+ 택시 1이 time=8에 시작해서 처음 운행하는 동안, 택시 2는 time=10에 차고를 나와서 두 번의 운행을 마쳤다.
+ 이 간단한 시뮬레이션에서, 마지막 이벤트가 time=110에 발생하면서 모든 이벤트가 기본 설정된 180분 안에 완료되었다.

예제 16-21은 콘솔을 이용해 클라이언트를 흉내낸 것이다. 

In [49]:
""" [예제 16-21] taxi_process() 코투린 돌려보기 """
taxi = taxi_process(13, 2, 0)
next(taxi)

Event(time=0, proc=13, action='leave garage')

In [50]:
taxi.send(_.time + 7) # 콘솔에서 _는 마지막 값에 바인딩된다.

Event(time=7, proc=13, action='pick up passenger')

In [51]:
taxi.send(_.time + 23)

Event(time=30, proc=13, action='drop off passenger')

In [52]:
taxi.send(_.time + 5)

Event(time=35, proc=13, action='pick up passenger')

In [53]:
taxi.send(_.time + 48)

Event(time=83, proc=13, action='drop off passenger')

In [54]:
taxi.send(_.time + 1)

Event(time=84, proc=13, action='going home')

In [55]:
taxi.send(_.time + 10)

StopIteration: 